# EWX API

In [26]:
import pandas as pd
import numpy as np
import json
import os
import time
import math
import datetime
import requests
import pprint
from pandas.io.json import json_normalize
pp = pprint.PrettyPrinter(indent = 4)

## Requests Header

Header requires a bearer token (expires every hour), namespace & response type.

In [88]:
bearer = 'ya29.ImCRB-enD_1PIqZDGC2t_6BNEHPfOicX4dbGZzJwruMR8F6usEEKae_u2BXzvzCvJWAFB-i7hf3zRkwMOB_D9LxTlikJeGMjq4MM7AbuODaEzcUsnauU4P81mch8yhRoAhc'
auth = ''.join(['Bearer ', bearer])
header = {'Authorization':auth, 'X-NAMESPACE':'na.engie.com', 'accept':'application/json'}

## User Get Request

In [89]:
url = "https://api.cloud.energyworx.com/_ah/api/ewx/v1/user/get"
user = requests.get(url, headers = header).json()

output = {'user_email':user['email'], 'namespace':user['namespace'], 'refresh': user['refreshToken']}
print('found user info:')
pp.pprint(output)
print('')

namespaces = user['namespaces']
for n in namespaces:
    print('found namespace: {}.'.format(n['name']))

found user info:
{   'namespace': 'na.engie.com',
    'refresh': '1/LEbljcUTdPixexfxu-n6Fw045FKqbN1ralhNSmdgEfeihp2RMRKpHCK027jVyulV',
    'user_email': 'steven.hurwitt@external.engie.com'}

found namespace: live.engie-na.com.
found namespace: na.engie-test.com.
found namespace: na.engie.com.


## Datasource Get Request

Function to parse properties & classifiers given datasource

In [102]:
def get_datasource(account):
    
    url = "https://api.cloud.energyworx.com/_ah/api/ewx/v1/datasource/get"
    ds_url = "/".join([url, account])

    datasource = requests.get(ds_url, headers = header).json()

    tags = datasource['tags']
    properties = []

    for t in tags:
        props = t['properties']
        for p in props:
            properties.append((p['key'], p['value']))

    properties = dict(properties)
    props_df = pd.DataFrame.from_dict(properties, orient = 'index', columns = ['properties'])

    channels = datasource['channels']
    classifiers = []

    for c in channels:
        classifiers.append((c['id'], c['classifier']))
    
    classifiers = dict(classifiers)
    class_df = pd.DataFrame.from_dict(classifiers, orient = 'index', columns = ['classifiers'])
    
    return(props_df, class_df)

In [103]:
properties, classifiers = get_datasource("NEPOOL_MECO_1286952002")

In [106]:
properties.head()

,properties
calculated,False
default,True
estimated,False
periodstatus,S
utilityprovided,True


In [107]:
classifiers.head()

,classifiers
DELIVERY_SCALAR,DELIVERY_SCALAR
DELIVERY_SCALAR_HEARTBEAT,DELIVERY_SCALAR_HEARTBEAT
TOTAL_DEMAND,TOTAL_DEMAND
TOTAL_DEMAND_HEARTBEAT,TOTAL_DEMAND_HEARTBEAT
DELIVERY_IDR,DELIVERY_IDR


## Query Get Request

In [ ]:
url = "/ewx/v1/query/execute"
eql = requests.get(url, headers = header)

In [ ]:
query = """SELECT *
FROM flow_metadata
WHERE STRUCT (datasource_id, flow_timestamp) IN
  (SELECT AS STRUCT datasource_id, MAX(flow_timestamp) AS flow_timestamp 
  FROM flow_metadata
  GROUP BY datasource_id)

AND datasource_id IN (
  SELECT datasource_id
  FROM tags, UNNEST(properties) as props
  WHERE tag = 'metadata'
  AND props.key = 'market'
  AND (props.value = 'NEPOOL' OR props.value = 'NYISO')
)
ORDER BY flow_timestamp DESC"""